In [ ]:
!pip install cryptography
!pip install sha3

In [ ]:
# import libraries
from secrets import randbits
from sympy import Matrix
from sympy.abc import x
import random
import hashlib
import sha3
from functools import reduce
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
from cryptography.fernet import Fernet

## Architecture Parameters

In [ ]:
# defining the nodes
master = {}
racks_c = {}

In [ ]:
# architecture parameters
a = 1
b = 15
replication_factor = 5
rack_system = {}
public_keys = {}
private_keys = {}

In [ ]:
# keys for racks
consts = {}
for i in range(1, 16, 1):
  consts[i] = [randbits(16) for _ in range(256)]

In [ ]:
# rack intelligence system
def get_racks():
  racks = random.sample(range(a, b+1), replication_factor)
  print('racks generated')
  return racks

## CRYSTALS Kyber & Hashing

In [ ]:
def generate_global_key_pair():

  def get_random_poly():
      """Generate a random polynomial."""
      coeffs = [randbits(16) for _ in range(256)]
      return coeffs

  def evaluate_poly(poly, point):
      """Evaluate a polynomial at a given point."""
      return sum(coeff * point ** i for i, coeff in enumerate(poly))

  def generate_key_pair():
      """Generate a public/private key pair."""
      # Generate a random polynomial
      private_key = get_random_poly()

      # Evaluate the polynomial at 256 points to get the public key
      public_key = [evaluate_poly(private_key, point) % 3329 for point in range(256)]

      return public_key, private_key

  public_key, private_key = generate_key_pair()
  print('global keys generated')
  return public_key, private_key

In [ ]:
def generate_rack_key_pair(racks):
  """Generate a public/private key pair."""

  def evaluate_poly(poly, point):
    """Evaluate a polynomial at a given point."""
    return sum(coeff * point ** i for i, coeff in enumerate(poly))

  def get_private_key(racks):
    """Generate the private key based on racks"""
    private_key = [0 for _ in range(256)]
    for rack in racks:
      private_key = [(a+b)%(2**16) for a,b in zip(private_key, consts[rack])]
    return private_key

  # Generate a random polynomial
  private_key = get_private_key(racks)

  # Evaluate the polynomial at 256 points to get the public key
  public_key = [evaluate_poly(private_key, point) % 3329 for point in range(256)]

  # Return the public key and the polynomial (as the private key)
  print('rack keys generated')
  return public_key, private_key

In [ ]:
def get_nonce(racks, curr_rack):
  racks = [rack for rack in racks if rack!=curr_rack]
  nonce = [sum(x) for x in zip(racks)]
  nonce = [x % (2**16) for x in nonce]
  print('rack nonce generated')
  return nonce

In [ ]:
def get_global_nonce(racks):
  keys = []
  for rack in racks:
    keys.append(consts[rack])
  nonce = [sum(x) for x in zip(*keys)]
  fnonce = [x % (2**16) for x in nonce]
  print('global nonce generated')
  return fnonce

In [ ]:
def get_rackcrypt(racks):
  randsums = [(sum(consts[rack]) % (2**16)) for rack in racks]
  product = reduce(lambda x, y: x*y, randsums) % (2**16)
  print('rackcrypt generated')
  return product

In [ ]:
def get_sha(msg):
  return hashlib.sha256(msg).hexdigest()

In [ ]:
def encapsulate_key(public_key, nonce):
    """Encapsulate a shared secret using the public key."""

    def evaluate_poly(poly, point):
      """Evaluate a polynomial at a given point."""
      return sum(coeff * point ** i for i, coeff in enumerate(poly))

    # Evaluate the polynomial at 256 points to get the shared secret
    shared_secret = [evaluate_poly(nonce, point) % 3329 for point in range(256)]

    # Evaluate the public key at each point and add the corresponding coefficient of the polynomial
    error_vector = [(evaluate_poly(public_key, point) + s) % 3329 for point, s in enumerate(shared_secret)]

    # Return the shared secret and the error vector
    print('key encapsulated')
    return shared_secret, error_vector

In [ ]:
def encode(public_key, nonce, message):
  key, _= encapsulate_key(public_key, nonce)
  key = (bin(sum(key) % (2**16))[2:])
  encrypted_data = int(key) | int(message)
  print('aes done')
  return encrypted_data

In [ ]:
def update_master(uid):
  rackcrypt = get_rackcrypt(rack_system[uid])
  sha1 = get_sha(bin(rackcrypt)[2:].encode('utf-8'))
  sha2 = get_sha(bin(int(sha1, 16))[2:].encode('utf-8'))
  hash = int(str(bin(int(sha2, 16))[2:])+str(bin(rackcrypt)[2:])+str(bin(int(sha1, 16))[2:]))
  encrypted = encode(global_public_key, global_nonce, hash)
  master[uid] = encrypted
  print('master updated')

In [ ]:
def store(uid, message):
  racks = rack_system[uid]
  sha1 = get_sha(bin(message)[2:].encode('utf-8'))
  sha2 = get_sha(bin(int(sha1, 16))[2:].encode('utf-8'))
  hash = int(str(bin(int(sha2, 16))[2:])+str(bin(message)[2:])+str(bin(int(sha1, 16))[2:]))

  for rack in racks:
    nonce = get_nonce(racks, rack)
    public_key = public_keys[uid]
    encrypted = encode(public_key, nonce, hash)
    racks_c['c{}'.format(rack)] = encrypted
  print('stored')

## User End

In [ ]:
# user's racks
def set_user_racks(uid):
  returning = True
  if rack_system.get(uid) is None:
    rack_system[uid] = get_racks()
    returning = False
  return returning, rack_system[uid]
  print('racks set')

# user's keys
def set_keys(uid):
  keys =  generate_rack_key_pair(rack_system[uid])
  public_keys[uid] = keys[0]
  private_keys[uid] = keys[1]
  print('keys set')

# generate global keys
global_public_key, global_private_key = generate_global_key_pair()
global_nonce = get_global_nonce(list(range(1, 16, 1)))

global keys generated
global nonce generated


In [ ]:
def main():

  # input user details
  uid = int(input('Enter User ID: '))

  # get user's racks
  returning, racks = set_user_racks(uid)
  if returning == False:
    set_keys(uid)
    update_master(uid)


  msg = input(('Enter message: '))
  bitmsg = ''
  for c in msg:
    bitmsg += bin(ord(c))[2:].zfill(8)
  sha1 = get_sha(bin(int(bitmsg))[2:].encode('utf-8'))
  sha2 = get_sha(bin(int(sha1, 16))[2:].encode('utf-8'))
  message = int(str(bin(int(sha2, 16))[2:])+
                str(bin(int(bitmsg))[2:])+
                str(bin(int(sha1, 16))[2:]))

  store(uid, message)

In [ ]:
main()

Enter User ID: 0
racks generated
rack keys generated
keys set
rackcrypt generated
key encapsulated
aes done
master updated
Enter message: hello supriti
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
stored


In [ ]:
rack_system

{0: [5, 1, 7, 12, 2]}

In [ ]:
master

{0: 11110100011011110011110000101111001000000010101100011100011011000001110010000001100011111111101010110001100100000111111110010101001110111011010001101111010000101010001101001100010110110110001011011101111111001010101010110110011001000110010110011100011110011011111000010100110101010111101010001000111001101101010111101011000011011101111001011111010111100001100110000100011111011111001011011110010000001100011110000010000110001101010110011010100000111011101100011100010001101110001011110100110101011011100000001001008906013506926}

In [ ]:
racks_c

{'c5': 110010000001010001100101110011011100111011101001011000111101100100111011110010011001000000000010010111000101010110001110111101110001111001001100011011101111001000001011110110100010000101110000011000100001011001101100011101010110001111011001001011101000111010001101001010110100010001101000011101101110011101110011111001011101011001100101001011011100111001001110110010000100000010101001011110001011101100000100001010100011100000010010111110100100001111100010010000001111011011001111100100010000011001110001101010001011011001101101001110001010111110011110111000001101001100010110001011111110101100111100000000100110111100000010001000110000111100110101110011010010100011111111101011110111001001110001111100101110110101101111110110010100100100100010100001101010001001001110110101101101111100011111010111000101010101010100110000000001101011000101010001001001101011110110001111110100011001000100101100010101010011111010110111000000000111110110101000100011101100100101011000110000110001100100011101010